In [1]:
# Import the pandas library and alias it as 'pd' for ease of use
import pandas as pd

# Import the numpy library and alias it as 'np' for ease of use
import numpy as np

In [ ]:
# Read the data from a CSV file named "orders.csv" into a pandas DataFrame named 'df'
df = pd.read_csv("orders.csv")


In [ ]:
# Filter the DataFrame 'df' to select rows where the 'Status' column is "COMPLETE"
# and the 'Product' column is "MIS"
df = df[(df["Status"] == "COMPLETE") & (df["Product"] == "MIS")]

In [ ]:
# Remove the 'Time' column from the DataFrame 'df'
df.drop(columns="Time", inplace=True)

# Split each value in the 'Qty.' column by '/' and take the first part,
# then convert the result to integers and replace the original 'Qty.' column
df["Qty."] = df["Qty."].apply(lambda x: x.split("/")[0]).astype(int)

## 1) Different types of charges for Individual trade

In [2]:
# Calculate the turnover for each row by multiplying the 'Qty.' and 'Avg. price' columns
df["Turnover"] = df["Qty."] * df["Avg. price"]

# Calculate the brokerage for each row as 0.03% of the turnover, rounded to 2 decimal places, with a minimum of 20
df["Brokerage"] = round(df["Turnover"] * 0.03 / 100, 2).apply(lambda x: 20 if x > 20 else x)

# Calculate the STT/CTT for each row as 0.025% of the turnover, rounded to 2 decimal places
df["STT/CTT"] = round(df["Turnover"] * 0.025 / 100, 2)

# For rows where the 'Type' column is 'BUY', set the STT/CTT to 0
df.loc[df['Type'] == 'BUY', 'STT/CTT'] = 0

# Calculate the ETC for each row as 0.00325% of the turnover, rounded to 2 decimal places
df["ETC"] = round(df["Turnover"] * 0.00325 / 100, 2)

# Calculate the SEBI charges for each row as (10/10000000) of the turnover, rounded to 2 decimal places
df["SEBI"] = round(10 / 10000000 * df["Turnover"], 2)

# Calculate the GST for each row as 18% of the total brokerage, SEBI charges, and ETC, rounded to 2 decimal places
df["GST"] = round(((df["Brokerage"] + df["SEBI"] + df["ETC"]) * 18) / 100, 2)

# Calculate the stamp charges for each row as 0.003%

,Type,Instrument,Product,Qty.,Avg. price,Status,Turnover,Brokerage,STT/CTT,ETC,SEBI,GST,Stamp Charges,Total Charges
1,BUY,ASHOKLEY,MIS,1000,125.70,COMPLETE,125700.0,20,0.00,4.09,0.13,4.36,3.77,32.35
3,BUY,TATAMOTORS,MIS,250,490.55,COMPLETE,122637.5,20,0.00,3.99,0.12,4.34,3.68,32.13
5,SELL,TATAMOTORS,MIS,250,492.10,COMPLETE,123025.0,20,30.76,4.00,0.12,4.34,0.00,59.22
7,SELL,ASHOKLEY,MIS,1000,125.96,COMPLETE,125960.0,20,31.49,4.09,0.13,4.36,0.00,60.07
8,BUY,ASHOKLEY,MIS,2000,125.70,COMPLETE,251400.0,20,0.00,8.17,0.25,5.12,7.54,41.08
9,SELL,ASHOKLEY,MIS,2000,125.95,COMPLETE,251900.0,20,62.98,8.19,0.25,5.12,0.00,96.54
11,SELL,ASHOKLEY,MIS,2000,125.95,COMPLETE,251900.0,20,62.98,8.19,0.25,5.12,0.00,96.54
12,BUY,ASHOKLEY,MIS,1000,125.60,COMPLETE,125600.0,20,0.00,4.08,0.13,4.36,3.77,32.34
13,BUY,ASHOKLEY,MIS,1000,125.65,COMPLETE,125650.0,20,0.00,4.08,0.13,4.36,3.77,32.34


## 2) Stock wise and Type wise analysis with weighted Avg. price & calculated charges

In [3]:
# Group the DataFrame 'df' by the columns 'Instrument' and 'Type', and perform aggregations on other columns
stock_type_wise = df.groupby(['Instrument', 'Type']).agg({
    'Qty.': 'sum',                # Sum of 'Qty.' for each group
    'Avg. price': 'mean',         # Mean of 'Avg. price' for each group
    'Turnover': 'sum',            # Sum of 'Turnover' for each group
    'Brokerage': 'sum',           # Sum of 'Brokerage' for each group
    'STT/CTT': 'sum',             # Sum of 'STT/CTT' for each group
    'ETC': 'sum',                 # Sum of 'ETC' for each group
    'SEBI': 'sum',                # Sum of 'SEBI' for each group
    'GST': 'sum',                 # Sum of 'GST' for each group
    'Stamp Charges': 'sum',       # Sum of 'Stamp Charges' for each group
    'Total Charges': 'sum'        # Sum of 'Total Charges' for each group
}).reset_index()                  # Reset the index to make it a regular DataFrame


,Instrument,Type,Qty.,Avg. price,Turnover,Brokerage,STT/CTT,ETC,SEBI,GST,Stamp Charges,Total Charges
0,ASHOKLEY,BUY,5000,125.662500,628350.0,80,0.00,20.42,0.64,18.20,18.85,138.11
1,ASHOKLEY,SELL,5000,125.953333,629760.0,60,157.45,20.47,0.63,14.60,0.00,253.15
2,TATAMOTORS,BUY,250,490.550000,122637.5,20,0.00,3.99,0.12,4.34,3.68,32.13
3,TATAMOTORS,SELL,250,492.100000,123025.0,20,30.76,4.00,0.12,4.34,0.00,59.22


## 3) Overall Summary of each Stocks

In [4]:
# Group the DataFrame 'df' by 'Instrument' and sum up the 'Total Charges' for each instrument
df1 = df.groupby('Instrument')['Total Charges'].sum().reset_index()

# Get unique shares/instruments from the DataFrame
shares = df1['Instrument'].unique()

# Create an empty dictionary 'd' to store the difference between buy and sell turnovers for each share
d = {}

# Iterate over each share/instrument
for i in shares:
    # Calculate the total turnover for buys of the current share
    buy = df[(df['Instrument'] == i) & (df['Type'] == 'BUY')]['Turnover'].sum()
    # Calculate the total turnover for sells of the current share
    sell = df[(df['Instrument'] == i) & (df['Type'] == 'SELL')]['Turnover'].sum()
    # Calculate the difference between sell and buy turnovers and store it in the dictionary
    d[i] = sell - buy

# Add the 'Gross PnL' column to df1, representing the difference between sell and buy turnovers for each share
df1["Gross PnL"] = d.values()

# Calculate the 'Net PnL' by subtracting the 'T

,Instrument,Total Charges,Gross PnL,Net PnL,% Charges on Gross PnL
0,ASHOKLEY,391.26,1410.0,1018.74,27.75
1,TATAMOTORS,91.35,387.5,296.15,23.57


In [5]:
# Open an ExcelWriter object to write data to an Excel file named "Kite problem assisment.xlsx"
with pd.ExcelWriter("Kite problem assisment.xlsx") as writer:
    # Write the 'df' DataFrame to a sheet named "Individual trade charge" without including the index
    df.to_excel(writer, sheet_name="Individual trade charge", index=False)
    
    # Write the 'stock_type_wise' DataFrame to a sheet named "stock_type_wise_charge" without including the index
    stock_type_wise.to_excel(writer, sheet_name="stock_type_wise_charge", index=False)
    
    # Write the 'df1' DataFrame to a sheet named "overall_summary" without including the index
    df1.to_excel(writer, sheet_name="overall_summary", index=False)